# Variabili categoriche

Una variabile categorica rappresenta valori discreti.

In [1]:
import pandas as pd 

df = pd.read_stata("../Data/rent99.dta")
df = df.astype({'location': 'int32'})

Prendiamo in considerazione la variabile categorica `location`

In [2]:
# Vediamo le categorie distinte di location
df['location'].value_counts()

1    1794
2    1210
3      78
Name: location, dtype: int64

Quando lavoriamo con variabili categoriche, non possiamo fare semplicemente una regressione lineare, perchè verrebbe trattata come variabile numerica.

**DA NON FARE!!!**

In [3]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

y = df['rentsqm']
X = df['location']
X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                rentsqm   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     24.05
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           9.88e-07
Time:                        17:01:28   Log-Likelihood:                -7104.8
No. Observations:                3082   AIC:                         1.421e+04
Df Residuals:                    3080   BIC:                         1.423e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.5439      0.124     52.911      0.0

**COSA FARE**

Dobbiamo creare delle variabili **dummy**.

Per farlo aggiungiamo colonne al dataframe in base al numero delle categorie e impostiamo il valore a 1 se appartiene alla categoria, altrimenti lo impostiamo a 0.

Durante la regressione dobbiamo togliere una varibile categorica di riferimento, questo perchè avere sia l'intercetta che tutte le variabili di riferimento, porta ad avere una matrice di Design che non è più a rango pieno e quindi non più invertibile.

> Cambiare variabile omessa, non cambia il fit del modello.
> 
> Tuttavia, si omette quella centrale o quella più frequente

Per fare la regressione specificando che una variabile è categorica utilizziamo la libreria `statsmodels.formula.api`.

La dichiarazione è la seguente:

```python
model = smf.ols(formula='y ~ C(nome_variabile_categorica)', data=nome_dataframe)
```


In [4]:
import statsmodels.formula.api as smf

model = smf.ols(formula='rentsqm ~ C(location)', data=df)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                rentsqm   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     13.77
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           1.11e-06
Time:                        17:01:28   Log-Likelihood:                -7103.0
No. Observations:                3082   AIC:                         1.421e+04
Df Residuals:                    3079   BIC:                         1.423e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            6.9565      0.057  

Se vogliamo aggiungere anche altre covariate utilizziamo la seguente sintassi:

```python
model = smf.ols(formula='y ~ C(nome_variabile_categorica) + covariata1 + ...', data=nome_dataframe)
```

In [5]:
model = smf.ols(formula='rentsqm ~ C(location) + area', data=df)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                rentsqm   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     159.5
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           4.12e-96
Time:                        17:01:28   Log-Likelihood:                -6894.1
No. Observations:                3082   AIC:                         1.380e+04
Df Residuals:                    3078   BIC:                         1.382e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            9.3553      0.125  

## Interpretazione

### Rimozione di una variabile dummy

I coefficienti delle variabili dummy sono **differenze** con la categoria omessa.

Ad esempio:

Avendo `alocation`, `glocation`, `tlocation` come variabili categoriche e prendendo come riferimento `alocation`,
il modello sopra indica che `rentsqm` a parità delle altre covariate (nel nostro caso caso solo `area`) è maggiore di 0.4646 se abbiamo `glocation` rispetto ad avere `alocation`, mentre è maggiore di 1.7537 se abbiamo `tlocation` rispetto a `alocation`.

### Rimozione dell'intercetta

Rimuovendo l'interetta al posto di una delle variabili dummy, i coefficienti mi indicano il **valore atteso** per ogni categoria, a parità delle altre covariate.
> Non sono più differenze, ma **livelli**

In [6]:
model = smf.ols(formula='rentsqm ~ 0 + C(location)', data=df)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                rentsqm   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     13.77
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           1.11e-06
Time:                        17:01:28   Log-Likelihood:                -7103.0
No. Observations:                3082   AIC:                         1.421e+04
Df Residuals:                    3079   BIC:                         1.423e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
C(location)[1]     6.9565      0.057    121.

# Interazioni tra variabili

Un'interazione è il prodotto tra due variabili.

Per eseguire un interazione usiamo il seguente comando:

```python
model = smf.ols(formula='y ~ C(var1) * C(var2)', data=df)
```

> Usare le interazioni se si crede che coppie di varibili si influenzino a vicenda o sono altamente correlate tra loro.
>
> L'interazione si usa solo a **coppie**, ma il numero di interazioni nel modello non è limitata.

In [9]:
# crea una nuova variabile che indica se la location è 2 o 3
df['gtlocation'] = ((df['location'] == 2) | (df['location'] == 3)).astype(int)
print(df['gtlocation'].value_counts())

model = smf.ols(formula='rentsqm ~ C(gtlocation) * C(cheating)', data=df)
results = model.fit()
print(results.summary())

0    1794
1    1288
Name: gtlocation, dtype: int64
                            OLS Regression Results                            
Dep. Variable:                rentsqm   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     135.3
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           2.48e-82
Time:                        17:02:33   Log-Likelihood:                -6925.9
No. Observations:                3082   AIC:                         1.386e+04
Df Residuals:                    3078   BIC:                         1.388e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------

## Interpretazione

L'interazione in sè per sè non ha interpretazione, è importante sapere che:

1. Se utilizziamo il modello con interazione **NON** possiamo più interpretare i coefficienti delle covariate coinvolte nell'interazione.

2. L'intercetta si riferisce alla base line del modello, cioè alla variabile dummy omessa. (In questo caso il volore al metro quadro di una casa avarege che non ha il riscaldamento centralizzato)